In [1]:
import numpy as np
import time
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
device = "cuda:3" if torch.cuda.is_available() else "cpu"
model = LlamaForCausalLM.from_pretrained(model_path, device_map=device)
tokenizer = LlamaTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
input_text = "'Once upon a time"
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
# 生成文本并保存 KV 缓存
outputs = model.generate(inputs, max_new_tokens=50, use_cache=True, return_dict_in_generate=True, output_scores=True)


In [ ]:
context_ids = inputs[:, :-1]
next_word_ids = inputs[:, -1:]

In [ ]:
inputs
context_ids
next_word_ids

In [ ]:
outputs

In [ ]:
generated_ids = outputs.sequences
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text)

In [114]:
past_key_values = outputs.past_key_values

In [ ]:
past_key_values

In [ ]:
past_key_values[0][0].shape[2]

In [116]:
output = model.generate(input_ids=next_word_ids, past_key_values=past_key_values)

RuntimeError: upper bound and larger bound inconsistent with step sign

In [ ]:
# 继续生成文本
input_text_continued = "And they come across a huge monster"
inputs_continued = tokenizer(input_text_continued, return_tensors="pt").to(device)
inputs_continued

In [ ]:
# 继续生成文本并传递以前的 KV 缓存
outputs_continued = model.generate(
    **inputs_continued,
    max_new_tokens=50,
    use_cache=True,
    return_dict_in_generate=True,
    output_scores=True,
    past_key_values=past_key_values  # 传递以前的 KV 缓存
)

# 解码生成的文本
generated_text = tokenizer.decode(outputs_continued.sequences[0], skip_special_tokens=True)
print(generated_text)

In [2]:
input_text = "'Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

In [3]:
out = model.generate(input_ids,  max_new_tokens=50, use_cache=True, return_dict_in_generate=True)
past_key_values = out.past_key_values
generated_ids = out.sequences

# Now we can continue generation using cache and already generated tokens
out_continued = model.generate(generated_ids,  max_new_tokens=50,past_key_values=past_key_values, return_dict_in_generate=True)
continued_generated_ids = out_continued.sequences

In [4]:
generated_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
continued_output = tokenizer.batch_decode(continued_generated_ids, skip_special_tokens=True)

In [5]:
generated_output

["'Once upon a time, in a land far, far away...' is a classic opening line used to transport readers to a magical world of wonder and enchantment. But what if the story takes place in a world that is not so far away? What if"]

In [6]:
continued_output

["'Once upon a time, in a land far, far away...' is a classic opening line used to transport readers to a magical world of wonder and enchantment. But what if the story takes place in a world that is not so far away? What if the magic and wonder are hidden in plain sight, waiting to be discovered?\n\nIn this collection of short stories, we explore the idea of magic and wonder in the everyday world. From the secret garden hidden behind a dumpster to the mag"]